# Battle of cities

A data science project to find the best candidate city for a new software R&D office.

In [109]:
import numpy as np
import pandas as pd
import requests
import folium
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

# Climate

## Gathering climate data from Wikipedia
We will fetch climate data from Wikipedia for US and European cities. We'll put them into clusters, and visualize the resulting clusters. It will help getting an idea what the seasons are like in U.S. cities for a European audience.

In [127]:
wikiurl = 'https://en.wikipedia.org/wiki/'

def getAvgTempData(continent, country=None):
    df = pd.read_html(f'{wikiurl}List_of_cities_by_average_temperature', match = continent)[0]
    if country:
        df = df[df['Country'] == country]
    df.drop(['Country', 'Ref.'], axis=1, inplace=True)
    df.set_index('City', inplace=True)
    return df

us_avg_temp = getAvgTempData('North America', 'United States')
print(us_avg_temp.index)
us_avg_temp.head()

Index(['Albuquerque', 'Anchorage', 'Atlanta', 'Austin', 'Baltimore', 'Boise',
       'Boston', 'Charlotte', 'Chicago', 'Columbus', 'Dallas', 'Denver',
       'Detroit', 'El Paso', 'Fairbanks', 'Houston', 'Indianapolis',
       'Jacksonville', 'Kansas City', 'Las Vegas', 'Lake Havasu City',
       'Los Angeles', 'Louisville', 'Memphis', 'Miami', 'Milwaukee',
       'Minneapolis', 'Nashville', 'New Orleans', 'New York City',
       'OklahomaCity', 'Omaha', 'Philadelphia', 'Phoenix', 'Pittsburgh',
       'Portland (OR)', 'Sacramento', 'Salt LakeCity', 'San Antonio',
       'San Diego', 'SanFrancisco', 'Seattle', 'St. Louis', 'Tampa', 'Tucson',
       'VirginiaBeach', 'Washington,D.C.', 'Wichita'],
      dtype='object', name='City')


,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Year
City,,,,,,,,,,,,,
Albuquerque,2.4(36.4),5.2(41.4),8.9(48.1),13.3(56.0),18.7(65.6),23.8(74.9),25.7(78.3),24.6(76.2),20.7(69.3),14.2(57.5),7.2(44.9),2.4(36.3),14.0(57.2)
Anchorage,−8.3(17.1),−6.6(20.2),−3.0(26.6),2.7(36.8),8.8(47.8),12.9(55.2),14.9(58.8),13.7(56.7),9.2(48.6),1.6(34.8),−5.4(22.2),−7.2(19.0),2.8(37.1)
Atlanta,6.3(43.3),8.4(47.1),12.3(54.2),16.4(61.5),21.1(69.9),25.2(77.4),26.9(80.5),26.3(79.4),22.9(73.2),17.1(62.8),12.0(53.6),7.3(45.1),16.8(62.3)
Austin,10.8(51.5),12.8(55.0),16.5(61.7),20.7(69.2),24.8(76.6),27.9(82.2),29.4(85.0),29.9(85.8),26.7(80.0),21.8(71.2),16.1(61.0),11.4(52.5),20.7(69.3)
Baltimore,0.8(33.5),2.4(36.4),6.8(44.2),12.4(54.3),17.6(63.6),22.8(73.0),25.3(77.6),24.3(75.7),20.2(68.4),13.7(56.7),8.3(47.0),2.9(37.3),13.1(55.6)


Clearly we'll need some additional data wrangling: the data values are strings, and what's worse, they include temperatures in both Celsius and Fahrenheit (in parentheses). As the target audience is European, only Celsius should remain and we should convert the strings into floats. Also some names are missing spaces (e.g. "OklahomaCity"), we need to fix those as well for geolocation to work later on.

In [128]:
def delete_imperial_measures(df):
    return df.replace(to_replace='([0-9,\.]+)\((.*)\)', value=r'\1', regex=True)

def wiki_df_to_float(df):
    return df.replace({'−': '-'}, regex=True).astype(float)

us_avg_temp = delete_imperial_measures(us_avg_temp)
us_avg_temp = wiki_df_to_float(us_avg_temp)

us_avg_temp.rename(index={'OklahomaCity': 'Oklahoma City', 'Portland (OR)': 'Portland, OR',
                          'Salt LakeCity': 'Salt Lake City', 'SanFrancisco': 'San Francisco',
                          'VirginiaBeach': 'Virginia Beach'}, inplace=True)

print(us_avg_temp.index)
us_avg_temp.head()



Index(['Albuquerque', 'Anchorage', 'Atlanta', 'Austin', 'Baltimore', 'Boise',
       'Boston', 'Charlotte', 'Chicago', 'Columbus', 'Dallas', 'Denver',
       'Detroit', 'El Paso', 'Fairbanks', 'Houston', 'Indianapolis',
       'Jacksonville', 'Kansas City', 'Las Vegas', 'Lake Havasu City',
       'Los Angeles', 'Louisville', 'Memphis', 'Miami', 'Milwaukee',
       'Minneapolis', 'Nashville', 'New Orleans', 'New York City',
       'Oklahoma City', 'Omaha', 'Philadelphia', 'Phoenix', 'Pittsburgh',
       'Portland, OR', 'Sacramento', 'Salt Lake City', 'San Antonio',
       'San Diego', 'San Francisco', 'Seattle', 'St. Louis', 'Tampa', 'Tucson',
       'Virginia Beach', 'Washington,D.C.', 'Wichita'],
      dtype='object', name='City')


,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Year
City,,,,,,,,,,,,,
Albuquerque,2.4,5.2,8.9,13.3,18.7,23.8,25.7,24.6,20.7,14.2,7.2,2.4,14.0
Anchorage,-8.3,-6.6,-3.0,2.7,8.8,12.9,14.9,13.7,9.2,1.6,-5.4,-7.2,2.8
Atlanta,6.3,8.4,12.3,16.4,21.1,25.2,26.9,26.3,22.9,17.1,12.0,7.3,16.8
Austin,10.8,12.8,16.5,20.7,24.8,27.9,29.4,29.9,26.7,21.8,16.1,11.4,20.7
Baltimore,0.8,2.4,6.8,12.4,17.6,22.8,25.3,24.3,20.2,13.7,8.3,2.9,13.1


Much better. A quick sanity check to see we got what we wanted:

In [129]:
us_avg_temp['Jan'].mean()

3.7208333333333337

In [130]:
eu_avg_temp = getAvgTempData('Europe')
eu_avg_temp = delete_imperial_measures(eu_avg_temp)
eu_avg_temp = wiki_df_to_float(eu_avg_temp)

In [131]:
eu_avg_temp.head()

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Year
City,,,,,,,,,,,,,
Tirana,6.7,7.8,10.0,13.4,18.0,21.6,24.0,23.8,20.7,16.0,11.7,8.1,15.2
Vienna,0.3,1.5,5.7,10.7,15.7,18.7,20.8,20.2,15.4,10.2,5.1,1.1,10.4
Minsk,-4.5,-4.4,0.0,7.2,13.3,16.4,18.5,17.5,12.1,6.6,0.6,-3.4,6.7
Brussels,3.3,3.7,6.8,9.8,13.6,16.2,18.4,18.0,14.9,11.1,6.8,3.9,10.5
Sarajevo,-0.5,1.4,5.7,10.0,14.8,17.7,19.7,19.7,15.3,11.0,5.4,0.9,10.1


In [132]:
eu_avg_temp.index

Index(['Tirana', 'Vienna', 'Minsk', 'Brussels', 'Sarajevo', 'Sofia', 'Zagreb',
       'Nicosia', 'Prague', 'Copenhagen', 'Tallinn', 'Helsinki', 'Kuopio',
       'Oulu', 'Marseille', 'Paris', 'Berlin', 'Frankfurt', 'Athens',
       'Piraeus', 'Budapest', 'Reykjavík', 'Dublin', 'Milan', 'Rome', 'Riga',
       'Vaduz', 'Vilnius', 'Luxembourg City', 'Skopje', 'Valletta', 'Chișinău',
       'Monaco', 'Podgorica', 'Amsterdam', 'Bergen', 'Oslo', 'Tromsø',
       'Warsaw', 'Lisbon', 'Bucharest', 'Arkhangelsk', 'Moscow', 'Murmansk',
       'Rostov-on-Don', 'Saint Petersburg', 'Sochi', 'Belgrade', 'Bratislava',
       'Ljubljana', 'Barcelona', 'Las Palmas de Gran Canaria', 'Madrid',
       'Seville', 'Stockholm', 'Zürich', 'Istanbul', 'Kiev', 'Lviv', 'Odessa',
       'Edinburgh', 'London'],
      dtype='object', name='City')

A quick discussion with stakeholders reveals that they especially like some other European cities' climate. Examples are Malaga, Luzern, Nice and Dubrovnik. Unfortunately the dataframe we fetched do not have those cities. However, the individual pages of the cities do have that information. Therefore we put together a new function for fetching climate data from those pages.

In [133]:
def getClimateData(city):    
    cityurl = city.replace(' ', '_')
    df = pd.read_html(f'{wikiurl}{cityurl}', match ='Climate data', header=1, index_col='Month')[0]
    df.index.rename('Stat', inplace=True)
    df = delete_imperial_measures(df)
    #delete rows that do not contain numbers
    df = df.apply(lambda x: pd.to_numeric(x, errors='coerce')).dropna()
    df = df.rename(index=lambda x: x.replace(' (°F)',''))
    return df

nice = getClimateData('Nice')

In [134]:
nice

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Year
Stat,,,,,,,,,,,,,
Record high °C,22.5,25.8,26.1,26.0,30.3,36.8,37.0,37.7,33.9,29.9,25.4,22.0,37.7
Average high °C,13.1,13.4,15.2,17.0,20.7,24.3,27.3,27.7,24.6,21.0,16.6,13.8,19.6
Daily mean °C,9.2,9.6,11.6,13.6,17.4,20.9,23.8,24.1,21.0,17.4,12.9,10.0,16.0
Average low °C,5.3,5.9,7.9,10.2,14.1,17.5,20.3,20.5,17.3,13.7,9.2,6.3,12.4
Average precipitation mm (inches),69.0,44.7,38.7,69.3,44.6,34.3,12.1,17.8,73.1,132.8,103.9,92.7,733.0
Average precipitation days (≥ 1.0 mm),5.8,4.7,4.6,7.1,5.2,3.8,1.8,2.4,4.9,7.2,7.2,6.4,61.2
Average snowy days,0.4,0.6,0.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,1.2
Mean monthly sunshine hours,157.7,171.2,217.5,224.0,267.1,306.1,347.5,315.8,242.0,187.0,149.3,139.3,2724.2
Average ultraviolet index,1.0,2.0,4.0,5.0,7.0,8.0,8.0,7.0,5.0,3.0,2.0,1.0,4.0


We only need the row 'Daily mean °C'. Append this to our dataframe:

In [135]:
eu_avg_temp = eu_avg_temp.append(nice.loc['Daily mean °C'].rename('Nice'))

Let's write a function for doing this to the remaining three cities mentioned above.

In [136]:
def append_to_df_city_climate_data(df, cities, stat_name):
    for city in cities:
        city_df = getClimateData(city)
        df = df.append(city_df.loc[stat_name].rename(city))
    return df

In [137]:
eu_avg_temp = append_to_df_city_climate_data(eu_avg_temp, ['Malaga', 'Luzern', 'Dubrovnik'], 'Daily mean °C')
eu_avg_temp

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Year
City,,,,,,,,,,,,,
Tirana,6.7,7.8,10.0,13.4,18.0,21.6,24.0,23.8,20.7,16.0,11.7,8.1,15.2
Vienna,0.3,1.5,5.7,10.7,15.7,18.7,20.8,20.2,15.4,10.2,5.1,1.1,10.4
Minsk,-4.5,-4.4,0.0,7.2,13.3,16.4,18.5,17.5,12.1,6.6,0.6,-3.4,6.7
Brussels,3.3,3.7,6.8,9.8,13.6,16.2,18.4,18.0,14.9,11.1,6.8,3.9,10.5
Sarajevo,-0.5,1.4,5.7,10.0,14.8,17.7,19.7,19.7,15.3,11.0,5.4,0.9,10.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
London,4.3,4.5,6.9,8.7,12.1,15.1,17.3,17.0,14.3,10.9,7.2,4.7,10.3
Nice,9.2,9.6,11.6,13.6,17.4,20.9,23.8,24.1,21.0,17.4,12.9,10.0,16.0
Malaga,12.1,12.9,14.7,16.3,19.3,23.0,25.5,26.0,23.5,19.5,15.7,13.2,18.5


Excellent! Now we have all cities we need.

## Create clusters
We will apply K-Means Clustering to form groups of cities. Seeing which European cities are in the same cluster as a candidate U.S. city, we'll get an idea about climate in that city.

In [138]:
kclusters = 6

city_avg_temp_clustering = pd.concat([eu_avg_temp.drop('Year', 1), us_avg_temp.drop('Year', 1)])
city_avg_temp_clustering

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
City,,,,,,,,,,,,
Tirana,6.7,7.8,10.0,13.4,18.0,21.6,24.0,23.8,20.7,16.0,11.7,8.1
Vienna,0.3,1.5,5.7,10.7,15.7,18.7,20.8,20.2,15.4,10.2,5.1,1.1
Minsk,-4.5,-4.4,0.0,7.2,13.3,16.4,18.5,17.5,12.1,6.6,0.6,-3.4
Brussels,3.3,3.7,6.8,9.8,13.6,16.2,18.4,18.0,14.9,11.1,6.8,3.9
Sarajevo,-0.5,1.4,5.7,10.0,14.8,17.7,19.7,19.7,15.3,11.0,5.4,0.9
...,...,...,...,...,...,...,...,...,...,...,...,...
Tampa,15.9,17.3,19.5,22.1,25.7,27.8,28.2,28.3,27.5,24.4,20.6,17.2
Tucson,11.6,13.1,15.8,19.7,24.6,29.6,30.8,29.8,27.7,21.9,15.6,11.3
Virginia Beach,4.9,6.1,9.8,14.8,19.4,24.3,26.7,25.7,22.6,16.9,11.8,6.9


In [139]:
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(city_avg_temp_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

array([4, 3, 5, 3, 3, 0, 3, 1, 3, 3, 5, 5, 2, 2, 4, 3, 3, 3, 1, 1, 0, 5,
       3, 0, 4, 5, 3, 5, 3, 0, 1, 0, 4, 4, 3, 3, 5, 5, 3, 4, 0, 2, 5, 2,
       0, 5, 4, 0, 0, 0, 4, 1, 4, 1, 5, 3, 4, 5, 5, 0, 3, 3, 4, 1, 3, 4,
       0, 2, 4, 1, 0, 0, 0, 4, 0, 0, 1, 0, 0, 4, 2, 1, 0, 1, 0, 1, 1, 4,
       0, 4, 1, 0, 5, 4, 1, 0, 4, 0, 0, 1, 0, 3, 4, 0, 1, 4, 4, 3, 0, 1,
       1, 4, 4, 0])

Let's put these clusters on a map!

In [ ]:
# Let's grab city coordinates
def do_geocode(geolocator, address):
    try:
        return geolocator.geocode(address)
    except GeocoderTimedOut:
        return do_geocode(geolocator, address)

coordinates = []
geolocator = Nominatim(user_agent="world_explorer")
for city in city_avg_temp_clustering.index:    
    location = do_geocode(geolocator, city)    
    coordinates.append([location.latitude, location.longitude])

coordinates

In [157]:
# create map
map_clusters = folium.Map(location=[40, -40], zoom_start=3)
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(np.asarray(coordinates)[:,0], np.asarray(coordinates)[:,1], city_avg_temp_clustering.index, kmeans.labels_):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Interesting results. **Cluster 0** is Central-Eastern Europe and many U.S. cities from Salt Lake City through New York until Boston. **Cluster 1** is the South-Mediterranean from Seville, Malaga on the West to Nicosia on the East, plus the Canaries. In the U.S. most southern cities (except California) also fall into this cluster which might be surprising.
**Cluster 2** is snow and ice: Oulu, Murmansk, Arhangelsk in Europe, Alaska in the U.S.
**Cluster 3**: A large part of Western Europe belongs here, the UK, Northern France, Germany. Interesting that very few cities in the U.S. have this climate: only Seattle and Portland in the North-West. **Cluster 4** is nice weather: the North Mediterranean plus Istanbul, Lisbon and Madrid in Europe, California, and the cities north to Cluster 1 and south to Cluster 0 in the US from El Paso to Virginia Beach. The **last cluster (#5)** is Oslo, Stockholm plus other cold Northeastern parts of Europe: the Baltic states, Ukraine, Belarus and Moscow. Not surprising that the U.S. doesn't have many entries here: only Minneapolis.

# Quality of life
We'll be examining the The **Movehub City Rankings** dataset, fetched from **Kaggle** (https://www.kaggle.com/blitzr/movehub-city-rankings).  This dataset has valuable features such as Health Care index, Pollution index and a more general and ambitious-sounding “Quality of Life” index for major cities around the world. We’ll just concentrate on U.S. cities.


In [167]:
qol = pd.read_csv('Coursera_Capstone/movehubqualityoflife.csv')
qol.set_index('City', inplace=True)
qol.head()

,Movehub Rating,Purchase Power,Health Care,Pollution,Quality of Life,Crime Rating
City,,,,,,
Caracas,65.18,11.25,44.44,83.45,8.61,85.70
Johannesburg,84.08,53.99,59.98,47.39,51.26,83.93
Fortaleza,80.17,52.28,45.46,66.32,36.68,78.65
Saint Louis,85.25,80.40,77.29,31.33,87.51,78.13
Mexico City,75.07,24.28,61.76,18.95,27.91,77.86


Let's join our previous U.S. dataset containing the monthly average temperatures with this dataset.

In [169]:
us_cities = pd.merge(qol, us_avg_temp.drop('Year', 1), left_index=True, right_index=True)
us_cities

,Movehub Rating,Purchase Power,Health Care,Pollution,Quality of Life,Crime Rating,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
City,,,,,,,,,,,,,,,,,,
Detroit,70.63,73.81,63.05,83.45,50.99,76.69,-3.7,-2.3,2.7,9.4,15.2,20.7,23.0,22.1,17.9,11.2,5.2,-1.2
Las Vegas,84.88,80.46,66.52,59.73,60.50,70.57,8.8,11.1,15.0,19.1,24.7,29.9,33.2,32.1,27.6,20.4,13.1,8.2
Philadelphia,83.31,68.77,54.17,38.64,65.53,68.58,0.5,2.0,6.4,12.2,17.7,22.9,25.6,24.7,20.6,14.2,8.7,3.0
Los Angeles,86.86,62.75,68.61,75.20,62.82,65.74,14.1,14.3,14.8,16.0,17.4,19.0,20.8,21.2,20.9,19.2,16.5,14.1
Miami,84.43,57.79,64.44,22.45,74.77,64.85,19.9,21.1,22.4,24.2,26.5,28.1,28.8,28.9,28.2,26.4,23.7,21.2
Houston,85.24,71.96,79.44,65.21,74.08,61.72,11.5,13.3,16.9,20.7,24.8,27.7,28.9,29.0,26.3,21.7,16.6,12.3
Tampa,84.27,75.72,56.94,49.89,80.75,60.73,15.9,17.3,19.5,22.1,25.7,27.8,28.2,28.3,27.5,24.4,20.6,17.2
Dallas,84.71,76.18,69.99,83.89,80.56,59.90,7.7,9.9,14.2,18.6,23.3,27.4,29.7,29.8,25.6,19.8,13.7,8.4
Baltimore,84.28,78.78,72.08,77.98,82.00,58.85,0.8,2.4,6.8,12.4,17.6,22.8,25.3,24.3,20.2,13.7,8.3,2.9


Nice. We immediately notice the Pollution value 0.00 for Nashville. It can't be right, it seems this data point is missing. Let's replace it with the mean Pollution value.

In [174]:
us_cities.loc['Nashville']['Pollution'] = us_cities['Pollution'].mean()
us_cities.loc['Nashville']['Pollution']

50.2395652173913

Let's sort our dataset by Quality of Life:

In [176]:
us_cities.sort_values('Quality of Life', ascending=False, inplace=True)
us_cities.head()

,Movehub Rating,Purchase Power,Health Care,Pollution,Quality of Life,Crime Rating,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
City,,,,,,,,,,,,,,,,,,
Austin,84.86,69.22,73.61,28.84,86.51,42.50,10.8,12.8,16.5,20.7,24.8,27.9,29.4,29.9,26.7,21.8,16.1,11.4
San Antonio,83.76,74.78,60.97,59.19,84.88,51.41,11.0,13.1,16.8,20.8,25.0,28.1,29.3,29.7,26.6,21.8,16.2,11.6
Charlotte,84.46,77.18,72.08,67.05,84.39,30.21,5.1,7.2,11.3,15.8,20.3,24.7,26.4,25.8,22.2,16.3,11.1,6.3
Seattle,85.38,78.46,75.46,32.90,84.10,42.03,5.4,6.2,7.9,10.1,13.2,15.9,18.6,18.8,16.2,11.4,7.4,4.7
Minneapolis,83.47,69.91,62.35,77.94,83.79,40.36,-9.1,-6.2,0.4,8.6,15.1,20.4,23.2,21.7,16.7,9.4,0.9,-6.8


Climatewise Minneapolis is in the cold Cluster 5, which our stakeholders likely wouldn't like, while San Antonio and Austin (Cluster 1) with their South-Mediterranean climate is great for holidays, but maybe not-so-much for productive work. This leaves us with Charlotte and Seattle: but Charlotte's air pollution value of 67.05 is not ideal. So, looks like our candidate in this first iteration of the project is... Seattle!

Before we move to selecting ideal neighborhoods in Seattle, let's investigate this "Quality of Life" index a bit. Can we find a relationship with our other data features? We'll run a quick and decidedly simple multiple regression to find out.

In [189]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
scale=StandardScaler()
lm = LinearRegression()

# Our predictors, let's normalize them before fitting the model, so we can use 
# the coefficient magnitudes as indicators of effect size
X = us_cities[['Purchase Power', 'Health Care', 'Pollution', 'Crime Rating', 'Jan', 'Apr', 'Jul', 'Oct']]
X_scaled = scale.fit_transform(X)

# Our target
y = us_cities['Quality of Life']

#fit model
lm.fit(X_scaled, y)

#Print results
print('Intercept: ', lm.intercept_)
print('Coefficients: ', lm.coef_)
print('R-squared:', lm.score(X_scaled, y))

Intercept:  77.40608695652173
Coefficients:  [ 0.63834069  2.36640877 -0.48216095 -6.02313142 -6.84068855 10.35034581
 -2.81065026 -0.68619446]
R-squared: 0.6342897332646185


R-squared is only 0.63, even in this in-sample evaluation of our model, so looks like there are other factors in play as well. The data we gathered so far is not enough to explain the Quality of Life scores. A later iteration may involve sourcing other data vectors as well. Still, looking at the coefficients produced by the model fitting, interestingly the January and April mean temperatures have the biggest effect among our predictors, and - not so surprisingly - the Crime Rate is important as well.

# Finding the right neighborhood
We'll be using the Foursquare API to get information about the neighborhoods of our candidate city.